In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import os
import gc

%matplotlib inline

pd.set_option('display.max_rows', 304)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Some notes/ideas from EDA:
* Looks like missing values were replaced with 0 (including in target!).
* Lag features? Feature for number of time_ids since this investment_id started? Lag of target itself?
* Note: if doing lag features, will need to be careful to combine test data with train data in order to make sure lag features are included.
* All features are scaled already (including target) - roughly mean of 0 and standard deviation 1. However, many features are bimodal, or skewed.
* All features have low correlation with target, but there are some that correlate with each other.
* Initial validation: time-based
* Validation goal: want validation score to be similar to leaderboard score

In [2]:
df = pd.read_parquet("../input/ubiquant-parquet/train_low_mem.parquet")

In [3]:
# for now, just use a sub-sample to do some exploration
#df = df.sample(frac=0.05, random_state=26)

# Validation Set

Initial strategy will be to hold out the highest ~25% of data in terms of time_id as a validation set

In [4]:
features = ['f_'+str(i) for i in range(300)]
val_time_cutoff=975
X_train = df.loc[df['time_id']<=val_time_cutoff, features].values
Y_train = df.loc[df['time_id']<=val_time_cutoff, 'target'].values
X_val = df.loc[df['time_id']>val_time_cutoff, features].values
Y_val = df.loc[df['time_id']>val_time_cutoff, 'target'].values

In [5]:
del df
gc.collect()

25

# Training

In [6]:
from sklearn.linear_model import SGDRegressor
from scipy.stats import pearsonr
import random

In [7]:
lr = SGDRegressor()
lr.fit(X_train, Y_train)
print ('Validation Pearson r: {}'.format(str(pearsonr(lr.predict(X_val), Y_val)[0])))

Validation Pearson r: 0.08100226457954703


# Final Train on Train+Val

In [8]:
X = np.concatenate([X_train, X_val])
Y = np.concatenate([Y_train, Y_val])
del X_train, Y_train
gc.collect()

42

In [9]:
lr = SGDRegressor()
lr.fit(X, Y)
print ('Validation Pearson r: {}'.format(str(pearsonr(lr.predict(X_val), Y_val)[0])))

Validation Pearson r: 0.06846588523565447


# Submission

In [10]:
import ubiquant
env = ubiquant.make_env()
iter_test = env.iter_test()
p = True
for (test_df, sample_prediction_df) in iter_test:
    if p:
        p = False
        print (sample_prediction_df.columns)
        print (test_df.columns)
    X_test = test_df[features].values
    sample_prediction_df['target'] = lr.predict(X_test)
    env.predict(sample_prediction_df) 

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
Index(['row_id', 'target'], dtype='object')
Index(['row_id', 'investment_id', 'f_0', 'f_1', 'f_2', 'f_3', 'f_4', 'f_5',
       'f_6', 'f_7',
       ...
       'f_290', 'f_291', 'f_292', 'f_293', 'f_294', 'f_295', 'f_296', 'f_297',
       'f_298', 'f_299'],
      dtype='object', length=302)
